In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/ATNLP/

Mounted at /content/drive
/content/drive/MyDrive/ATNLP


In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 13.1 MB/s 
     |████████████████████████████████| 184 kB 69.7 MB/s 
     |████████████████████████████████| 174 kB 51.9 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 
     |████████████████████████████████| 173 kB 86.9 MB/s 
     |████████████████████████████████| 168 kB 83.5 MB/s 
     |████████████████████████████████| 168 kB 69.2 MB/s 
     |████████████████████████████████| 166 kB 65.9 MB/s 
     |████████████████████████████████| 166 kB 63.1 MB/s 
     |████████████████████████████████| 162 kB 64.2 MB/s 
     |████████████████████████████████| 162 kB 72.7 MB/s 
     |████████████████████████████████| 158 kB 53.9 MB/s 
     |████████████████████████████████| 157 kB 78.9 MB/s 
     |████████████████████████████████| 157 kB 65.1 MB/s 
     |████████████████████████████████| 157 kB 66.7 MB/s 
     |███████████████████████████

In [ ]:
%cd transformer_scan/

/content/drive/MyDrive/ATNLP/transformer_scan


In [ ]:
import os
import json
import numpy as np
import random
from collections import Counter, defaultdict
from tqdm import tqdm
import time
import math

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pandas as pd
plt.style.use('seaborn-whitegrid')
plt.rcParams['figure.dpi'] = 300



import torch
import torch.nn as nn
import torch.optim as optim
from models.transformer_new import *

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
SOS_token = 0
EOS_token = 1
pad_idx = 2

class Format:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {SOS_token: "SOS", EOS_token: "EOS", pad_idx: "pad"}
        self.n_words = 3  

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.n_words += 1
            self.word2count[word] = 1
        else:
            self.word2count[word] += 1


def readFile(filename):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open(filename, encoding='utf-8').read().strip().split('\n')

    # exclude "IN: "
    pairs = [s[4:].split(' OUT: ') for s in lines]

    input_lang = Format("input")
    output_lang = Format("output")

    return input_lang, output_lang, pairs
  
def prepareData(filename):
    input_lang, output_lang, pairs = readFile(filename)
    print("Read %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


# obtain word indices in a sentence
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


# transform to tensor format and add a special token
def tensorFromSentence_input(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    # indexes.insert(0, SOS_token)
    # indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorFromSentence_output(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.insert(0, SOS_token)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

# for input and target (one-hot vectors)
def tensorsFromPair(pair):
    input_tensor = tensorFromSentence_input(input_lang, pair[0])
    target_tensor = tensorFromSentence_output(output_lang, pair[1])
    return (input_tensor, target_tensor)


def calculate_mean_std(acc_dict):
    mean = []
    error = []
    keys = sorted(acc_dict[0])
    num_runs = len(acc_dict)
    
    for key in keys:
        t = []
        for d in acc_dict:
            t.append(d[key])
        mean.append(np.mean(t))
        error.append(np.std(t) / np.sqrt(num_runs))
    return np.array(mean), np.array(error), keys


def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# input_lang, output_lang, pairs = prepareData("./data/scan/simple_split/tasks_train_simple.txt")
# input_lang_test, output_lang_test, pairs_test = prepareData("./data/scan/simple_split/tasks_test_simple.txt")

input_lang, output_lang, pairs = prepareData("./data/scan/simple_split/size_variations/tasks_train_simple_p32.txt")
input_lang_test, output_lang_test, pairs_test = prepareData("./data/scan/simple_split/size_variations/tasks_test_simple_p32.txt")


# input_lang, output_lang, pairs = prepareData("/Users/zhouyuqin/Desktop/ATNLP/SCAN/add_prim_split/tasks_train_addprim_jump.txt")
# input_lang_test, output_lang_test, pairs_test = prepareData("/Users/zhouyuqin/Desktop/ATNLP/SCAN/add_prim_split/tasks_test_addprim_jump.txt")



# input_lang, output_lang, pairs = prepareData("/Users/zhouyuqin/Desktop/ATNLP/SCAN/length_split/tasks_train_length.txt")
# input_lang_test, output_lang_test, pairs_test = prepareData("/Users/zhouyuqin/Desktop/ATNLP/SCAN/length_split/tasks_test_length.txt")



print(random.choice(pairs))
print(len(pairs), type(pairs))

Reading lines...
Read 6691 sentence pairs
Counting words...
Counted words:
input 16
output 9
Reading lines...
Read 14219 sentence pairs
Counting words...
Counted words:
input 16
output 9
['look opposite left thrice and jump left', 'I_TURN_LEFT I_TURN_LEFT I_LOOK I_TURN_LEFT I_TURN_LEFT I_LOOK I_TURN_LEFT I_TURN_LEFT I_LOOK I_TURN_LEFT I_JUMP']
6691 <class 'list'>


In [ ]:
# n_iters = 100000
# training_pairs = [tensorsFromPair(random.choice(pairs)) for i in range(n_iters)]
training_pairs = [tensorsFromPair(pair) for pair in pairs]
test_pairs = [tensorsFromPair(pair) for pair in pairs_test]

In [ ]:
len(training_pairs)

6691

In [ ]:
class Lang:
  ''' 
   ### Training and Data ###
    num_runs: Number of runs to do.
    num_epochs: Number of training epochs
    
    ### Models ### 
    d_model: Dimension of inputs/outputs in transformer
    nhead: Number of heads in transformer multihead attention
    num_encoder_layers: Number of layers in transformer encoder
    num_decoder_layers: Number of layers in transformer decoder
    dim_feedforward: Dimension of feedforward layers in transformer
    dropout: Dropout rate
              
    ### Optimization ### 
    learning_rate: Fixed learning rate for Adam optimizer


    ### Output options ###
    results_dir: Results subdirectory to save results
    out_data_file: Name of output data file with training loss data
    checkpoint_path: Path to output saved weights.
    checkpoint_every: Epochs before evaluating model and saving weights
    record_loss_every: iters before printing and recording loss
  '''

  def __init__(self, num_runs = 1, num_epochs = 30, d_model = 32, nhead = 8, num_decoder_layers = 2, 
               num_encoder_layers = 2,  d_feedforward = 256, learning_rate = 0.001,
               dropout = 0.1, checkpoint_every = 1, record_loss_every = 2000, clip = 5, teacher_forcing_ratio = 0):
    
    self.num_runs = num_runs
    self.num_epochs = num_epochs
    self.d_model = d_model
    self.nhead = nhead
    self.num_encoder_layers = num_encoder_layers
    self.num_decoder_layers = num_decoder_layers
    self.d_feedforward = d_feedforward
    self.learning_rate = learning_rate
    self.dropout = dropout
    self.checkpoint_every = checkpoint_every
    self.record_loss_every = record_loss_every
    self.clip = clip
    self.teacher_forcing_ratio = teacher_forcing_ratio

In [ ]:
args = Lang(d_feedforward = 512, nhead= 6)

In [ ]:
def train(name, model, args):              
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr = args.learning_rate)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(args.num_epochs):
        start_time = time.time()
        epoch_loss = 0
        print_loss_total = 0 

        for iter, batch in enumerate(training_pairs):
            loss = 0
            optimizer.zero_grad()

            ### Encoder ###
            src = batch[0].T
            src_mask = model.make_src_mask(src)
            enc_src = model.encoder(src, src_mask)
            
            ### Decoder ### 
            trg = batch[1].T
            trg_in = trg[:,:-1] ##[<SOS>, y_1, ..., y_2] 
            trg_out = trg[:,1:] ##[y_1,..., y_2, <EOS>]
            
            use_teacher_forcing = True if random.random() < args.teacher_forcing_ratio else False
            
            if not use_teacher_forcing:
                trg_mask = model.make_trg_mask(trg_in)
                output, _ = model.decoder(trg_in, enc_src, trg_mask, src_mask)       
                loss = criterion(output[0], trg_out.view(-1))    
                trg_indexes =  output[0].argmax(1)
            
            else:
                trg_indexes = [SOS_token]
                for i in range(trg_in.shape[1]):
                    trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)
                    trg_mask = model.make_trg_mask(trg_tensor)
                    output, _ = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
                    # only consider the prediction at time t 
                    output = output[0][i]
                    
                    ### gready search
                    pred_token = output.argmax(0).item()
                    trg_indexes.append(pred_token)
                    
                    ## calculate loss
                    loss += criterion(output, trg_out[0][i])

                    # break if the new term is <EOS> 
                    if pred_token == EOS_token:
                        break

                trg_indexes = trg_indexes[1:]
                loss = loss / trg_in.shape[1]

            
            loss.backward()
            wandb.log({"train/loss": loss.data.item(), "train/iter": iter})
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
            optimizer.step()
            
            # Record loss
            print_loss_total += loss.data.item()
            epoch_loss += loss.data.item()
            if (iter + 1) % args.record_loss_every == 0:
                print_loss_avg = print_loss_total / args.record_loss_every
                print(f'Epoch: {epoch} | Iter: {iter} | Loss: {print_loss_avg:.3f}') 
                print_loss_total = 0 
       
        torch.save(model.state_dict(), name)
        ### Recode loss
        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        epoch_loss_avg = epoch_loss / len(training_pairs)

        wandb.log({"train/epoch": epoch, "train/epoch_loss": epoch_loss_avg})
        print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {epoch_loss_avg:.3f}')


In [ ]:
# train(args)

In [ ]:
# def test_teacher(data, model):
#     model.eval()
#     with torch.no_grad():
#         all_correct_trials = [] # list of booleans indicating whether correct
        
#         for batch in data:
#             src = batch[0].T
#             trg = batch[1].T
#             out, _ = model(src, trg[:,:-1])

#             preds = torch.argmax(out, dim = 2)
#             correct_pred = preds == trg[:,1:]

#             correct_pred = correct_pred.cpu().numpy()
#             correct = correct_pred.all(0).tolist()
#             all_correct_trials += correct

#     accuracy = np.mean(all_correct_trials)
#     model.train()
#     return accuracy

# test_teacher(test_pairs, model)

In [ ]:
def translate_sentence(batch, model, max_len = 50):
    model.eval()
    src = batch[0].T
    src_mask = model.make_src_mask(src)
    with torch.no_grad():
        enc_src = model.encoder(src, src_mask)

    #
    trg_indexes = [SOS_token]
    for i in range(max_len):
        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)
        trg_mask = model.make_trg_mask(trg_tensor)
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        output = output[0][i]
        pred_token = output.argmax(0).item()
        trg_indexes.append(pred_token)
        #
        if pred_token == EOS_token:
            break
    return trg_indexes[1:], attention

def test_accuracy(data, model):
    all_correct_trials = 0
    for i, batch in enumerate(data):
      index, _  = translate_sentence(batch, model)

      trg = batch[1].T
      trg_out = trg[:,1:] ##[y_1,..., y_2, <EOS>]
      correct = trg_out[0].tolist() == index
      all_correct_trials += correct
        
      if (i+1)  % 10000 == 0:
          print(i, all_correct_trials/ i)
            
    return all_correct_trials / len(data)


In [ ]:
# test_accuracy(test_pairs, model)

In [ ]:
total_runs = 5
for run in range(total_runs):
  wandb.init(
      project="ATNLP", 
      name = f"exp_32p_{run}", 
      config={"epochs": 1,
              "name": f"./results/transformer/exp_32p_{run}"
      })
  
  # Copy your config 
  config = wandb.config

  model = Seq2Seq(Encoder(input_lang.n_words, 
                        args.nhead * 9, 
                        args.num_encoder_layers, 
                        args.nhead, 
                        args.d_feedforward, 
                        args.dropout, 
                        device), 
                    Decoder(output_lang.n_words, 
                            args.nhead * 9, 
                            args.num_decoder_layers, 
                            args.nhead, 
                            args.d_feedforward, 
                            args.dropout, 
                            device), pad_idx, pad_idx, device).to(device)
  train(config.name, model, args)
  
  print("Testing:")
  model.load_state_dict(torch.load(config.name))
  test_acc = test_accuracy(test_pairs, model)
  wandb.log({"test/accuracy": test_acc})
      
  # Mark the run as finished
  wandb.finish()

wandb: Currently logged in as: yuqinzhou. Use `wandb login --relogin` to force relogin


Epoch: 0 | Iter: 1999 | Loss: 1.062
Epoch: 0 | Iter: 3999 | Loss: 0.845
Epoch: 0 | Iter: 5999 | Loss: 0.794
Epoch: 01 | Time: 2m 30s
	Train Loss: 0.887
Epoch: 1 | Iter: 1999 | Loss: 0.740
Epoch: 1 | Iter: 3999 | Loss: 0.717
Epoch: 1 | Iter: 5999 | Loss: 0.700
Epoch: 02 | Time: 2m 30s
	Train Loss: 0.716
Epoch: 2 | Iter: 1999 | Loss: 0.676
Epoch: 2 | Iter: 3999 | Loss: 0.656
Epoch: 2 | Iter: 5999 | Loss: 0.652
Epoch: 03 | Time: 2m 26s
	Train Loss: 0.660
Epoch: 3 | Iter: 1999 | Loss: 0.636
Epoch: 3 | Iter: 3999 | Loss: 0.627
Epoch: 3 | Iter: 5999 | Loss: 0.633
Epoch: 04 | Time: 2m 28s
	Train Loss: 0.632
Epoch: 4 | Iter: 1999 | Loss: 0.618
Epoch: 4 | Iter: 3999 | Loss: 0.606
Epoch: 4 | Iter: 5999 | Loss: 0.611
Epoch: 05 | Time: 2m 29s
	Train Loss: 0.611
Epoch: 5 | Iter: 1999 | Loss: 0.593
Epoch: 5 | Iter: 3999 | Loss: 0.585
Epoch: 5 | Iter: 5999 | Loss: 0.580
Epoch: 06 | Time: 2m 26s
	Train Loss: 0.585
Epoch: 6 | Iter: 1999 | Loss: 0.581
Epoch: 6 | Iter: 3999 | Loss: 0.560
Epoch: 6 | Iter:

test/accuracy,▁
train/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/epoch_loss,█▆▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
train/iter,▃▇▅▄▁█▅▅▂▁▆▅▂▂▆▅▃▂▆▆▃▂▇▆▃▃▇▇▄▃█▇▄▄▁█▅▄▁█
train/loss,▇▅▇▅▂▃▆▃▂▇▂█▇▆▃▄▃▃▄▄▆▄▆▅▅▄▃▄▄▂▃▄▁▃▂▂▄▅▅▅
test/accuracy,0.11196
train/epoch,29
train/epoch_loss,0.39258
train/iter,6690
train/loss,0.31253


Epoch: 0 | Iter: 1999 | Loss: 1.051
Epoch: 0 | Iter: 3999 | Loss: 0.842
Epoch: 0 | Iter: 5999 | Loss: 0.798
Epoch: 01 | Time: 2m 34s
	Train Loss: 0.883
Epoch: 1 | Iter: 1999 | Loss: 0.748
Epoch: 1 | Iter: 3999 | Loss: 0.726
Epoch: 1 | Iter: 5999 | Loss: 0.710
Epoch: 02 | Time: 2m 28s
	Train Loss: 0.724
Epoch: 2 | Iter: 1999 | Loss: 0.682
Epoch: 2 | Iter: 3999 | Loss: 0.661
Epoch: 2 | Iter: 5999 | Loss: 0.669
Epoch: 03 | Time: 2m 30s
	Train Loss: 0.669
Epoch: 3 | Iter: 1999 | Loss: 0.647
Epoch: 3 | Iter: 3999 | Loss: 0.637
Epoch: 3 | Iter: 5999 | Loss: 0.632
Epoch: 04 | Time: 2m 30s
	Train Loss: 0.638
Epoch: 4 | Iter: 1999 | Loss: 0.621
Epoch: 4 | Iter: 3999 | Loss: 0.615
Epoch: 4 | Iter: 5999 | Loss: 0.610
Epoch: 05 | Time: 2m 29s
	Train Loss: 0.615
Epoch: 5 | Iter: 1999 | Loss: 0.599
Epoch: 5 | Iter: 3999 | Loss: 0.600
Epoch: 5 | Iter: 5999 | Loss: 0.604
Epoch: 06 | Time: 2m 30s
	Train Loss: 0.599
Epoch: 6 | Iter: 1999 | Loss: 0.586
Epoch: 6 | Iter: 3999 | Loss: 0.580
Epoch: 6 | Iter:

test/accuracy,▁
train/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/epoch_loss,█▅▅▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
train/iter,▃▇▅▄▁█▅▅▂▁▆▅▂▂▆▅▃▂▆▆▃▂▇▆▃▃▇▇▄▃█▇▄▄▁█▅▄▁█
train/loss,██▇▇▁▂█▆▂▂▁▅▆▅▂▃▄▄▃▃▄▃▇▄▅▄▃▃▂▂▆▄▄▃▂▁▂▅▁▃
test/accuracy,0.06752
train/epoch,29
train/epoch_loss,0.44193
train/iter,6690
train/loss,0.56053


...

test/accuracy,▁
train/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/epoch_loss,█▆▅▅▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
train/iter,▃▇▅▄▁█▅▅▂▁▆▅▂▂▆▅▃▂▆▆▃▂▇▆▃▃▇▇▄▃█▇▄▄▁█▅▄▁█
train/loss,▇▄▅▅▂▆▅▄▂▃▁▇▅▄▃▅█▄▅▃█▃▄▆▃▅█▃▄▂▂▃▃▃▂▂▂▂▁▃
test/accuracy,0.09987
train/epoch,29
train/epoch_loss,0.37572
train/iter,6690
train/loss,0.34966


Epoch: 0 | Iter: 1999 | Loss: 1.054
Epoch: 0 | Iter: 3999 | Loss: 0.838
Epoch: 0 | Iter: 5999 | Loss: 0.798
Epoch: 01 | Time: 2m 37s
	Train Loss: 0.884
Epoch: 1 | Iter: 1999 | Loss: 0.760
Epoch: 1 | Iter: 3999 | Loss: 0.731
Epoch: 1 | Iter: 5999 | Loss: 0.723
Epoch: 02 | Time: 2m 44s
	Train Loss: 0.735
Epoch: 2 | Iter: 1999 | Loss: 0.698
Epoch: 2 | Iter: 3999 | Loss: 0.682
Epoch: 2 | Iter: 5999 | Loss: 0.683
Epoch: 03 | Time: 2m 43s
	Train Loss: 0.685
Epoch: 3 | Iter: 1999 | Loss: 0.661
Epoch: 3 | Iter: 3999 | Loss: 0.648
Epoch: 3 | Iter: 5999 | Loss: 0.643
Epoch: 04 | Time: 2m 47s
	Train Loss: 0.649
Epoch: 4 | Iter: 1999 | Loss: 0.630
Epoch: 4 | Iter: 3999 | Loss: 0.619
Epoch: 4 | Iter: 5999 | Loss: 0.615
Epoch: 05 | Time: 2m 48s
	Train Loss: 0.621
Epoch: 5 | Iter: 1999 | Loss: 0.598
Epoch: 5 | Iter: 3999 | Loss: 0.604
Epoch: 5 | Iter: 5999 | Loss: 0.601
Epoch: 06 | Time: 2m 42s
	Train Loss: 0.601
Epoch: 6 | Iter: 1999 | Loss: 0.580
Epoch: 6 | Iter: 3999 | Loss: 0.583
Epoch: 6 | Iter:

test/accuracy,▁
train/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/epoch_loss,█▆▅▅▅▄▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁
train/iter,▃▇▅▄▁█▅▅▂▁▆▅▂▂▆▅▃▂▆▆▃▂▇▆▃▃▇▇▄▃█▇▄▄▁█▅▄▁█
train/loss,▇▇█▅▃▅▇▅▂▅▂▄▆▆▂▅▆▅▇▃▆▃▅▄▅▄▆▄▃▂▂▃▃▄▄▁▃▁▂▁
test/accuracy,0.16077
train/epoch,29
train/epoch_loss,0.35693
train/iter,6690
train/loss,0.18258


Epoch: 0 | Iter: 1999 | Loss: 1.064
Epoch: 0 | Iter: 3999 | Loss: 0.844
Epoch: 0 | Iter: 5999 | Loss: 0.807
Epoch: 01 | Time: 2m 47s
	Train Loss: 0.892
Epoch: 1 | Iter: 1999 | Loss: 0.777
Epoch: 1 | Iter: 3999 | Loss: 0.741
Epoch: 1 | Iter: 5999 | Loss: 0.731
Epoch: 02 | Time: 2m 44s
	Train Loss: 0.744
Epoch: 2 | Iter: 1999 | Loss: 0.704
Epoch: 2 | Iter: 3999 | Loss: 0.688
Epoch: 2 | Iter: 5999 | Loss: 0.684
Epoch: 03 | Time: 2m 45s
	Train Loss: 0.689
Epoch: 3 | Iter: 1999 | Loss: 0.655
Epoch: 3 | Iter: 3999 | Loss: 0.637
Epoch: 3 | Iter: 5999 | Loss: 0.646
Epoch: 04 | Time: 2m 46s
	Train Loss: 0.645
Epoch: 4 | Iter: 1999 | Loss: 0.629
Epoch: 4 | Iter: 3999 | Loss: 0.616
Epoch: 4 | Iter: 5999 | Loss: 0.616
Epoch: 05 | Time: 2m 45s
	Train Loss: 0.619
Epoch: 5 | Iter: 1999 | Loss: 0.597
Epoch: 5 | Iter: 3999 | Loss: 0.597
Epoch: 5 | Iter: 5999 | Loss: 0.604
Epoch: 06 | Time: 2m 49s
	Train Loss: 0.599
Epoch: 6 | Iter: 1999 | Loss: 0.588
Epoch: 6 | Iter: 3999 | Loss: 0.582
Epoch: 6 | Iter:

test/accuracy,▁
train/epoch,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
train/epoch_loss,█▆▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁
train/iter,▃▇▅▄▁█▅▅▂▁▆▅▂▂▆▅▃▂▆▆▃▂▇▆▃▃▇▇▄▃█▇▄▄▁█▅▄▁█
train/loss,▅▃▆█▁▂▄▆▂▃▁▄▄▃▂▂▃▂▃▂▄▂▄▃▃▃▃▃▃▁▃▃▂▃▁▁▃▄▂▂
test/accuracy,0.08875
train/epoch,29
train/epoch_loss,0.42077
train/iter,6690
train/loss,0.37871
